# Discrete Choice : Linear Models

## Definitions and Properties

### **Random Utility Model (RUM):**

Let $\mathbf{U}_i = \mathbf{v}_i + \boldsymbol{\varepsilon}_i$, where $\boldsymbol{\varepsilon}_i \sim \text{i.i.d. Gumbel}(0, 1)$.

The agent $i$ chooses alternative $y_i$ such that:

$$
y_i = \arg\max_{j} \ U_{ij}
$$

---

### **Conditional Logit** (alternative-specific features):

$$
\mathbf{v}_i = X \boldsymbol{\beta}, \quad X \in \mathbb{R}^{J \times K}, \ \boldsymbol{\beta} \in \mathbb{R}^{K}
$$

where:
- $X$ is the matrix of alternative features (same for all individuals),
- $\mathbf{v}_i \in \mathbb{R}^{J}$ is the deterministic utility vector for individual $i$,
- $J$ is the number of alternatives, $K$ is the number of features per alternative.

---

### **Multinomial Logit** (individual-specific features):

$$
\mathbf{v}_i = B \mathbf{x}_i, \quad B \in \mathbb{R}^{J \times K}, \ \mathbf{x}_i \in \mathbb{R}^{K}
$$

where:
- $B$ is the matrix of coefficients (one row per alternative),
- $\mathbf{x}_i$ is the feature vector of individual $i$ (same across alternatives),
- $\mathbf{v}_i \in \mathbb{R}^{J}$ is again the deterministic utility vector.

---

### **Conditional Choice Probability (CCP):**

$$
\Pr(y_i = d \mid \mathbf{v}_i) = \frac{\exp(v_{id})}{\sum_{j=1}^{J} \exp(v_{ij})}
$$


## Conditional Logit

In [ ]:
def conditional_logit(X_alt: np.ndarray, beta: np.ndarray) -> np.ndarray:
    pass

## Multinomial Logit

In [ ]:
def multinomial_logit(X_ind: np.ndarray, beta: np.ndarray) -> np.ndarray:
    pass
    